# Sleeper Fantasy Football Stats Analysis

Comprehensive statistical analysis of Hoyt Sleeper fantasy football league using DuckDB for local querying and exploration.

This notebook fetches matchup data from Hoyt Sleeper league and computes:
- Overall standings and win/loss records
- Top 10 highest and lowest individual week performances
- Scoring frequency (most high/low scores by week)
- Head-to-head performance metrics
- Average points in wins vs losses


## Setup: Import Libraries and Initialize Database


In [62]:
import requests
import pandas as pd
import duckdb
import json
from datetime import datetime
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Initialize DuckDB in-memory database
db = duckdb.connect(':memory:')


## Fetch League Data from Sleeper API


In [63]:
# Read league ID
LEAGUE_ID = 1255671107596333056
BASE_URL = "https://api.sleeper.app/v1"

# Fetch league information
league_response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}")
league_info = league_response.json()


## Fetch Users, Rosters, and Matchups


In [64]:
# Fetch users and rosters
users_response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/users")
users_list = users_response.json()
users_dict = {user['user_id']: user for user in users_list}

rosters_response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/rosters")
rosters_list = rosters_response.json()
rosters_dict = {roster['roster_id']: roster for roster in rosters_list}


In [65]:
# Fetch matchups for all weeks
matchups_by_week = {}

for week in range(1, 22):
    try:
        response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/matchups/{week}")
        if response.status_code == 200:
            matchups_list = response.json()
            if matchups_list:
                matchups_by_week[week] = matchups_list
            else:
                break
    except Exception as e:
        break


## Build DuckDB Tables


In [67]:
# Build matchup records
matchup_records = []

for week, matchups in matchups_by_week.items():
    for matchup in matchups:
        roster_id = matchup['roster_id']
        matchup_id = matchup['matchup_id']
        
        roster = rosters_dict.get(roster_id, {})
        owner_id = roster.get('owner_id')
        user = users_dict.get(owner_id, {})
        team_name = user.get('metadata', {}).get('team_name', f'Team_{roster_id}')
        
        points = matchup.get('points', 0)
        
        players = matchup.get('players', [])
        player_scores = {}
        for player_id in players:
            score = matchup.get('players_points', {}).get(player_id, 0)
            if score > 0:
                player_scores[player_id] = score
        
        if player_scores:
            top_player_id = max(player_scores, key=player_scores.get)
            top_player_score = player_scores[top_player_id]
        else:
            top_player_id = None
            top_player_score = 0
        
        matchup_records.append({
            'week': week,
            'roster_id': roster_id,
            'team_name': team_name,
            'matchup_id': matchup_id,
            'points': points,
            'top_player_id': top_player_id,
            'top_player_score': top_player_score,
            'players_json': json.dumps(player_scores)
        })

matchups_df = pd.DataFrame(matchup_records)
db.register('matchups_raw', matchups_df)


## Create Matchups Statistics Table


In [68]:
# Create matchups table with win/loss calculation
db.execute("DROP TABLE IF EXISTS matchups")

opponent_mapping = {}
for week, matchups in matchups_by_week.items():
    for matchup in matchups:
        roster_id = matchup['roster_id']
        matchup_id = matchup['matchup_id']
        
        key = (week, matchup_id, roster_id)
        opponent_mapping[key] = None

for week, matchups in matchups_by_week.items():
    for matchup in matchups:
        roster_id = matchup['roster_id']
        matchup_id = matchup['matchup_id']
        
        for other_matchup in matchups:
            if other_matchup['matchup_id'] == matchup_id and other_matchup['roster_id'] != roster_id:
                opponent_id = other_matchup['roster_id']
                opponent_roster = rosters_dict.get(opponent_id, {})
                opponent_owner_id = opponent_roster.get('owner_id')
                opponent_user = users_dict.get(opponent_owner_id, {})
                opponent_name = opponent_user.get('metadata', {}).get('team_name', f'Team_{opponent_id}')
                
                key = (week, matchup_id, roster_id)
                opponent_mapping[key] = opponent_name
                break

db.execute("""
    CREATE TABLE matchups AS
    SELECT 
        m.week,
        m.roster_id,
        m.team_name,
        m.matchup_id,
        m.points,
        m.top_player_id,
        m.top_player_score,
        m.players_json,
        CASE WHEN m.points > COALESCE(m2.points, 0) THEN 1 ELSE 0 END as won
    FROM matchups_raw m
    LEFT JOIN matchups_raw m2 ON m.week = m2.week 
        AND m.matchup_id = m2.matchup_id 
        AND m.roster_id != m2.roster_id
""")


## Statistical Analysis Results

Complete league statistics for weeks 1-14 (completed weeks only):


In [69]:
# Nothing to display here - just running queries for the final stats
pass


In [71]:
print("\n")
print("╔" + "═" * 98 + "╗")
print("║" + " FANTASY FOOTBALL STATS - WEEKS 1-14 ".center(98) + "║")
print("╚" + "═" * 98 + "╝")

# OVERALL STANDINGS
print("\n📊 OVERALL STANDINGS")
print("─" * 100)
standings = db.execute("""
SELECT 
    team_name,
    SUM(won) as wins,
    COUNT(*) - SUM(won) as losses,
    ROUND(AVG(points), 2) as avg_pts
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY wins DESC, avg_pts DESC
""").df()

for i, (_, row) in enumerate(standings.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['wins'])}-{int(row['losses']):<3}  {row['avg_pts']:>7.2f} avg pts")

# TOP 10 SCORES
print("\n\n🔥 TOP 10 SCORES")
print("─" * 100)
top_10 = db.execute("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY points DESC) as rank,
    team_name,
    week,
    ROUND(points, 2) as points,
    CASE WHEN won = 1 THEN 'W' ELSE 'L' END as result
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
ORDER BY points DESC
LIMIT 10
""").df()

for _, row in top_10.iterrows():
    print(f"  {int(row['rank']):2}. {row['team_name']:<30} Wk {int(row['week']):2}  {row['points']:>7.2f}  [{row['result']}]")

# TOP 10 LOWEST SCORES
print("\n\n❄️  TOP 10 LOWEST SCORES")
print("─" * 100)
low_10 = db.execute("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY points ASC) as rank,
    team_name,
    week,
    ROUND(points, 2) as points,
    CASE WHEN won = 1 THEN 'W' ELSE 'L' END as result
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
ORDER BY points ASC
LIMIT 10
""").df()

for _, row in low_10.iterrows():
    print(f"  {int(row['rank']):2}. {row['team_name']:<30} Wk {int(row['week']):2}  {row['points']:>7.2f}  [{row['result']}]")

# CONSISTENCY
print("\n\n🎯 CONSISTENCY SCORE (Lower = More Consistent)")
print("─" * 100)
consistency = db.execute("""
SELECT 
    team_name,
    ROUND(AVG(points), 2) as avg_pts,
    ROUND(STDDEV_POP(points), 2) as std_dev
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY std_dev ASC
""").df()

for i, (_, row) in enumerate(consistency.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['std_dev']:>6.2f} std dev")

# MOST HIGH SCORES BY WEEK
print("\n\n🏆 MOST HIGH SCORES BY WEEK")
print("─" * 100)
high_freq = db.execute("""
SELECT 
    team_name,
    COUNT(*) as times_highest
FROM (
    SELECT 
        week,
        team_name,
        points,
        ROW_NUMBER() OVER (PARTITION BY week ORDER BY points DESC) as rank
    FROM matchups
    WHERE week >= 1 AND week <= 14 AND points > 0
)
WHERE rank = 1
GROUP BY team_name
ORDER BY times_highest DESC
""").df()

for i, (_, row) in enumerate(high_freq.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['times_highest']):>2} times")

# MOST LOW SCORES BY WEEK
print("\n\n📉 MOST LOW SCORES BY WEEK")
print("─" * 100)
low_freq = db.execute("""
SELECT 
    team_name,
    COUNT(*) as times_lowest
FROM (
    SELECT 
        week,
        team_name,
        points,
        ROW_NUMBER() OVER (PARTITION BY week ORDER BY points ASC) as rank
    FROM matchups
    WHERE week >= 1 AND week <= 14 AND points > 0
)
WHERE rank = 1
GROUP BY team_name
ORDER BY times_lowest DESC
""").df()

for i, (_, row) in enumerate(low_freq.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['times_lowest']):>2} times")

# WINS VS LOWEST OPPONENT
print("\n\n⚡ MOST WINS VS LOWEST-SCORING OPPONENT")
print("─" * 100)
clutch = db.execute("""
WITH opponent_points AS (
    SELECT 
        m1.week,
        m1.matchup_id,
        m1.team_name,
        m1.points as my_points,
        m1.won,
        m2.points as opponent_points
    FROM matchups m1
    JOIN matchups m2 ON m1.week = m2.week 
        AND m1.matchup_id = m2.matchup_id 
        AND m1.roster_id != m2.roster_id
    WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
),
weekly_min AS (
    SELECT 
        week,
        MIN(opponent_points) as min_opponent_score
    FROM opponent_points
    GROUP BY week
)
SELECT 
    op.team_name,
    COUNT(*) as wins_vs_low_opp,
    ROUND(AVG(op.opponent_points), 2) as avg_opp_score
FROM opponent_points op
JOIN weekly_min wm ON op.week = wm.week AND op.opponent_points = wm.min_opponent_score
WHERE op.won = 1
GROUP BY op.team_name
ORDER BY wins_vs_low_opp DESC
""").df()

for i, (_, row) in enumerate(clutch.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['wins_vs_low_opp']):>2} wins")

# LOSSES VS HIGHEST OPPONENT
print("\n\n😰 MOST LOSSES VS HIGHEST-SCORING OPPONENT")
print("─" * 100)
tough = db.execute("""
WITH opponent_points AS (
    SELECT 
        m1.week,
        m1.matchup_id,
        m1.team_name,
        m1.points as my_points,
        m1.won,
        m2.points as opponent_points
    FROM matchups m1
    JOIN matchups m2 ON m1.week = m2.week 
        AND m1.matchup_id = m2.matchup_id 
        AND m1.roster_id != m2.roster_id
    WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
),
weekly_max AS (
    SELECT 
        week,
        MAX(opponent_points) as max_opponent_score
    FROM opponent_points
    GROUP BY week
)
SELECT 
    op.team_name,
    COUNT(*) as losses_vs_high_opp,
    ROUND(AVG(op.opponent_points), 2) as avg_opp_score
FROM opponent_points op
JOIN weekly_max wm ON op.week = wm.week AND op.opponent_points = wm.max_opponent_score
WHERE op.won = 0
GROUP BY op.team_name
ORDER BY losses_vs_high_opp DESC
""").df()

for i, (_, row) in enumerate(tough.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['losses_vs_high_opp']):>2} losses")

# AVG POINTS IN WINS
print("\n\n💪 AVG POINTS IN WINS")
print("─" * 100)
wins = db.execute("""
SELECT 
    team_name,
    ROUND(AVG(points), 2) as avg_pts_in_win
FROM matchups
WHERE won = 1 AND week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY avg_pts_in_win DESC
""").df()

for i, (_, row) in enumerate(wins.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['avg_pts_in_win']:>7.2f} avg")

# AVG POINTS IN LOSSES
print("\n\n😔 AVG POINTS IN LOSSES")
print("─" * 100)
losses = db.execute("""
SELECT 
    team_name,
    ROUND(AVG(points), 2) as avg_pts_in_loss
FROM matchups
WHERE won = 0 AND week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY avg_pts_in_loss DESC
""").df()

for i, (_, row) in enumerate(losses.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['avg_pts_in_loss']:>7.2f} avg")


# AVERAGE POINT MARGIN
print("\n\n📊 AVERAGE POINT MARGIN (vs Opponent)")
print("─" * 100)
avg_margin = db.execute("""
SELECT 
    m1.team_name,
    ROUND(AVG(m1.points - COALESCE(m2.points, 0)), 2) as avg_margin
FROM matchups m1
LEFT JOIN matchups m2 ON m1.week = m2.week 
    AND m1.matchup_id = m2.matchup_id 
    AND m1.roster_id != m2.roster_id
WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
GROUP BY m1.team_name
ORDER BY avg_margin DESC
""").df()

for i, (_, row) in enumerate(avg_margin.iterrows(), 1):
    margin = row['avg_margin']
    sign = "+" if margin > 0 else ""
    print(f"  {i:2}. {row['team_name']:<30} {sign}{margin:>6.2f}")

# MOST DOMINANT WINS
print("\n\n🏅 MOST DOMINANT WINS (Biggest Point Margins)")
print("─" * 100)
dominant = db.execute("""
SELECT 
    m1.team_name,
    m1.week,
    ROUND(m1.points - m2.points, 2) as margin,
    ROUND(m1.points, 2) as pts_for,
    ROUND(m2.points, 2) as pts_against
FROM matchups m1
JOIN matchups m2 ON m1.week = m2.week 
    AND m1.matchup_id = m2.matchup_id 
    AND m1.roster_id != m2.roster_id
WHERE m1.won = 1 AND m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
ORDER BY margin DESC
LIMIT 10
""").df()

for idx, (_, row) in enumerate(dominant.iterrows(), 1):
    print(f"  {idx:2}. {row['team_name']:<30} Wk {int(row['week']):2}  +{row['margin']:>6.2f}")

# CLOSEST MATCHUPS
print("\n\n🤝 CLOSEST MATCHUPS (Smallest Margins)")
print("─" * 100)
closest = db.execute("""
SELECT 
    m1.team_name,
    m2.team_name as opponent,
    m1.week,
    ROUND(ABS(m1.points - m2.points), 2) as margin,
    CASE WHEN m1.points > m2.points THEN 'W' ELSE 'L' END as result
FROM matchups m1
JOIN matchups m2 ON m1.week = m2.week 
    AND m1.matchup_id = m2.matchup_id 
    AND m1.roster_id != m2.roster_id
WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
ORDER BY margin ASC
LIMIT 10
""").df()

for idx, (_, row) in enumerate(closest.iterrows(), 1):
    print(f"  {idx:2}. {row['team_name']:<25} vs {row['opponent']:<25} {row['result']} by {row['margin']:>5.2f}")

# WEEK-TO-WEEK VOLATILITY
print("\n\n📈 WEEK-TO-WEEK VOLATILITY (Biggest Swings)")
print("─" * 100)
volatility = db.execute("""
WITH week_pairs AS (
    SELECT 
        t1.team_name,
        ROUND(ABS(t1.points - t2.points), 2) as swing
    FROM matchups t1
    JOIN matchups t2 ON t1.team_name = t2.team_name 
        AND t1.week + 1 = t2.week
    WHERE t1.week >= 1 AND t1.week < 14 AND t1.points > 0 AND t2.points > 0
)
SELECT 
    team_name,
    ROUND(AVG(swing), 2) as avg_swing,
    MAX(swing) as biggest_swing
FROM week_pairs
GROUP BY team_name
ORDER BY biggest_swing DESC
""").df()

for i, (_, row) in enumerate(volatility.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['biggest_swing']:>6.2f}")

# HEAD-TO-HEAD RECORDS
print("\n\n🎲 HEAD-TO-HEAD RECORDS")
print("─" * 100)
h2h = db.execute("""
WITH h2h_records AS (
    SELECT 
        m1.team_name,
        m2.team_name as opponent,
        SUM(CASE WHEN m1.points > m2.points THEN 1 ELSE 0 END) as wins,
        COUNT(*) as games
    FROM matchups m1
    JOIN matchups m2 ON m1.week = m2.week 
        AND m1.matchup_id = m2.matchup_id 
        AND m1.roster_id != m2.roster_id
    WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
    GROUP BY m1.team_name, m2.team_name
    ORDER BY m1.team_name, wins DESC
)
SELECT * FROM h2h_records
""").df()

current_team = None
for _, row in h2h.iterrows():
    if row['team_name'] != current_team:
        if current_team is not None:
            print()
        current_team = row['team_name']
        print(f"  {current_team}")
    losses = int(row['games']) - int(row['wins'])
    print(f"    vs {row['opponent']:<28} {int(row['wins'])}-{losses}")

print("\n╔" + "═" * 98 + "╗")
print("║" + " END OF STATS ".center(98) + "║")
print("╚" + "═" * 98 + "╝\n")




╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                               FANTASY FOOTBALL STATS - WEEKS 1-14                                ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝

📊 OVERALL STANDINGS
────────────────────────────────────────────────────────────────────────────────────────────────────
   1. Egbuka Matata                  11-3     140.46 avg pts
   2. Jefferson Starship             9-5     127.11 avg pts
   3. 🙌🏼 Praise Jah 🙌🏼               8-6     120.36 avg pts
   4. El Travador                    7-7     129.66 avg pts
   5. Luke at my Big Dicker          7-7     127.92 avg pts
   6. Spicy Bijan Mustard            7-7     126.21 avg pts
   7. Bridezilla                     7-7     115.74 avg pts
   8. Super Weenie Hut Jr            6-8     113.19 avg pts
   9. Kenergy                        4-10    122.97 avg pts
  10. Kittle Me This            

In [74]:
# LATE-ROUND STEALS
print("\n\n🌟 LATE-ROUND STEALS (Picks 101+)")
print("─" * 100)

# Fetch draft data
draft_response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/drafts")
drafts = draft_response.json()
draft_id = drafts[0]['draft_id']

picks_response = requests.get(f"{BASE_URL}/draft/{draft_id}/picks")
draft_picks = picks_response.json()

players_response = requests.get(f"{BASE_URL}/players/nfl")
players = players_response.json()

# Calculate player points
player_total_points = {}
for week in range(1, 22):
    response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/matchups/{week}")
    if response.status_code == 200:
        matchups = response.json()
        if not matchups:
            break
        for matchup in matchups:
            for player_id, points in matchup.get('players_points', {}).items():
                if points > 0:
                    player_total_points[player_id] = player_total_points.get(player_id, 0) + points

late_round_steals = []
for pick in draft_picks:
    if pick['pick_no'] and pick['pick_no'] > 100:
        player_id = pick['player_id']
        if player_id in players:
            player_name = players[player_id].get('full_name', f'Player {player_id}')
            pos = players[player_id].get('position', 'UNK')
        else:
            player_name = f'Player {player_id}'
            pos = 'UNK'
        
        total_points = player_total_points.get(player_id, 0)
        round_num = (pick['pick_no'] - 1) // 10 + 1
        
        # Get the team that drafted this player
        roster_id = pick.get('roster_id')
        team_name = "Unknown"
        if roster_id and roster_id in rosters_dict:
            roster = rosters_dict[roster_id]
            owner_id = roster.get('owner_id')
            if owner_id in users_dict:
                team_name = users_dict[owner_id].get('metadata', {}).get('team_name', f'Team_{roster_id}')
        
        if total_points > 0:
            late_round_steals.append({
                'pick_no': pick['pick_no'],
                'round': round_num,
                'player_name': player_name,
                'pos': pos,
                'total_points': total_points,
                'team_name': team_name
            })

late_round_steals = sorted(late_round_steals, key=lambda x: x['total_points'], reverse=True)
for i, steal in enumerate(late_round_steals[:10], 1):
    print(f"  {i:2}. Pick {steal['pick_no']:3} (Rd {steal['round']:2}) | {steal['team_name']:<30} | {steal['player_name']:<25} ({steal['pos']}) | {steal['total_points']:>6.2f} pts")

# ADP CONSISTENCY BY POSITION
print("\n\n📊 AVG PPR POINTS BY ADP BRACKET (By Position)")
print("─" * 100)
for position in ['QB', 'RB', 'WR', 'TE', 'K', 'DEF']:
    pos_picks = [p for p in draft_picks if p['player_id'] and p['player_id'] in players and players[p['player_id']].get('position') == position]
    
    if not pos_picks:
        continue
    
    early = [p for p in pos_picks if p['pick_no'] and p['pick_no'] <= 50]
    mid = [p for p in pos_picks if p['pick_no'] and 50 < p['pick_no'] <= 100]
    late = [p for p in pos_picks if p['pick_no'] and p['pick_no'] > 100]
    
    early_avg = sum(player_total_points.get(p['player_id'], 0) for p in early) / len(early) if early else 0
    mid_avg = sum(player_total_points.get(p['player_id'], 0) for p in mid) / len(mid) if mid else 0
    late_avg = sum(player_total_points.get(p['player_id'], 0) for p in late) / len(late) if late else 0
    
    print(f"\n  {position} (QB: 1-2 rounds, RB: 1-4 rounds, WR: 1-4 rounds, TE: 1-3 rounds)")
    print(f"    Picks 1-50:     {early_avg:>6.2f} avg ({len(early):2} picks)")
    print(f"    Picks 51-100:   {mid_avg:>6.2f} avg ({len(mid):2} picks)")
    print(f"    Picks 101+:     {late_avg:>6.2f} avg ({len(late):2} picks)")

# POSITIONAL PERFORMANCE
print("\n\n🏆 POSITIONAL PERFORMANCE (Total Draft Production)")
print("─" * 100)
pos_performance = {}
for pick in draft_picks:
    if pick['player_id'] and pick['player_id'] in players:
        player = players[pick['player_id']]
        pos = player.get('position', 'UNK')
        total_pts = player_total_points.get(pick['player_id'], 0)
        
        if pos not in pos_performance:
            pos_performance[pos] = []
        pos_performance[pos].append(total_pts)

for pos in sorted(pos_performance.keys()):
    points_list = pos_performance[pos]
    avg_pts = sum(points_list) / len(points_list)
    count = len(points_list)
    print(f"  {pos:3} | {count:2} players drafted, avg {avg_pts:>6.2f} pts/player")



🌟 LATE-ROUND STEALS (Picks 101+)
────────────────────────────────────────────────────────────────────────────────────────────────────
   1. Pick 160 (Rd 16) | Kenergy                        | Drake Maye                (QB) | 232.78 pts
   2. Pick 139 (Rd 14) | Spicy Bijan Mustard            | Justin Herbert            (QB) | 231.14 pts
   3. Pick 159 (Rd 16) | Spicy Bijan Mustard            | Player SEA                (DEF) | 165.80 pts
   4. Pick 136 (Rd 14) | Jefferson Starship             | Brandon Aubrey            (K) | 141.00 pts
   5. Pick 138 (Rd 14) | Luke at my Big Dicker          | Cameron Dicker            (K) | 139.00 pts
   6. Pick 128 (Rd 13) | Egbuka Matata                  | Keenan Allen              (WR) | 119.90 pts
   7. Pick 101 (Rd 11) | Kenergy                        | Cam Skattebo              (RB) | 115.70 pts
   8. Pick 131 (Rd 14) | Bridezilla                     | Player DEN                (DEF) | 115.60 pts
   9. Pick 147 (Rd 15) | El Travador            

In [76]:
# WORST VALUE PICKS
print("\n\n💸 WORST VALUE PICKS (High Pick, Low Production)")
print("─" * 100)

import statistics

# Group all picks by position and ADP bracket to calculate expected averages
pos_bracket_points = {}
for pick in draft_picks:
    if pick['player_id'] and pick['player_id'] in players:
        player = players[pick['player_id']]
        pos = player.get('position', 'UNK')
        pick_no = pick['pick_no']
        
        if pick_no and pick_no <= 100:
            bracket = 'early' if pick_no <= 50 else 'mid'
            key = (pos, bracket)
            points = player_total_points.get(pick['player_id'], 0)
            
            if key not in pos_bracket_points:
                pos_bracket_points[key] = []
            pos_bracket_points[key].append(points)

# Calculate expected points by position/bracket
expected_points = {}
for key, points_list in pos_bracket_points.items():
    if points_list:
        expected_points[key] = statistics.mean(points_list)

# Find biggest value misses
value_misses = []
for pick in draft_picks:
    if pick['player_id'] and pick['player_id'] in players:
        player = players[pick['player_id']]
        pos = player.get('position', 'UNK')
        pick_no = pick['pick_no']
        
        if pick_no and pick_no <= 100:
            bracket = 'early' if pick_no <= 50 else 'mid'
            key = (pos, bracket)
            
            if key in expected_points:
                actual_points = player_total_points.get(pick['player_id'], 0)
                expected = expected_points[key]
                value_miss = expected - actual_points
                
                if value_miss > 10:
                    player_name = player.get('full_name', f'Player {pick["player_id"]}')
                    round_num = (pick_no - 1) // 10 + 1
                    
                    # Get the team that drafted this player
                    roster_id = pick.get('roster_id')
                    team_name = "Unknown"
                    if roster_id and roster_id in rosters_dict:
                        roster = rosters_dict[roster_id]
                        owner_id = roster.get('owner_id')
                        if owner_id in users_dict:
                            team_name = users_dict[owner_id].get('metadata', {}).get('team_name', f'Team_{roster_id}')
                    
                    value_misses.append({
                        'pick_no': pick_no,
                        'round': round_num,
                        'player_name': player_name,
                        'pos': pos,
                        'actual': actual_points,
                        'expected': expected,
                        'miss': value_miss,
                        'team_name': team_name
                    })

value_misses = sorted(value_misses, key=lambda x: x['miss'], reverse=True)

for i, miss in enumerate(value_misses[:10], 1):
    print(f"  {i:2}. Pick {miss['pick_no']:3} (Rd {miss['round']:2}) | {miss['team_name']:<30} | {miss['player_name']:<25} ({miss['pos']}) | {miss['actual']:>6.2f} vs {miss['expected']:>6.2f} exp ({miss['miss']:>5.2f} below)")




💸 WORST VALUE PICKS (High Pick, Low Production)
────────────────────────────────────────────────────────────────────────────────────────────────────
   1. Pick  97 (Rd 10) | 🙌🏼 Praise Jah 🙌🏼               | Kyler Murray              (QB) |  61.68 vs 205.48 exp (143.80 below)
   2. Pick  39 (Rd  4) | Spicy Bijan Mustard            | James Conner              (RB) |  29.30 vs 169.73 exp (140.43 below)
   3. Pick  34 (Rd  4) | El Travador                    | Joe Burrow                (QB) |  58.46 vs 181.14 exp (122.68 below)
   4. Pick  77 (Rd  8) | 🙌🏼 Praise Jah 🙌🏼               | Kaleb Johnson             (RB) |   0.10 vs 108.05 exp (107.95 below)
   5. Pick  37 (Rd  4) | 🙌🏼 Praise Jah 🙌🏼               | Mike Evans                (WR) |  27.00 vs 128.76 exp (101.76 below)
   6. Pick  47 (Rd  5) | El Travador                    | Chuba Hubbard             (RB) |  78.80 vs 169.73 exp (90.93 below)
   7. Pick  40 (Rd  4) | Kenergy                        | Omarion Hampton           (RB)

In [75]:
# WORST DRAFT PICKS (BIGGEST BUSTS)
print("\n\n💔 WORST DRAFT PICKS (Biggest Busts)")
print("─" * 100)

busts = []
for pick in draft_picks:
    if pick['pick_no'] and pick['pick_no'] <= 100:
        if pick['player_id'] and pick['player_id'] in players:
            player = players[pick['player_id']]
            player_name = player.get('full_name', f'Player {pick["player_id"]}')
            pos = player.get('position', 'UNK')
            total_points = player_total_points.get(pick['player_id'], 0)
            round_num = (pick['pick_no'] - 1) // 10 + 1
            
            # Get the team that drafted this player
            roster_id = pick.get('roster_id')
            team_name = "Unknown"
            if roster_id and roster_id in rosters_dict:
                roster = rosters_dict[roster_id]
                owner_id = roster.get('owner_id')
                if owner_id in users_dict:
                    team_name = users_dict[owner_id].get('metadata', {}).get('team_name', f'Team_{roster_id}')
            
            busts.append({
                'pick_no': pick['pick_no'],
                'round': round_num,
                'player_name': player_name,
                'pos': pos,
                'total_points': total_points,
                'team_name': team_name
            })

# Sort by lowest points
busts = sorted(busts, key=lambda x: x['total_points'])

for i, bust in enumerate(busts[:10], 1):
    print(f"  {i:2}. {bust['team_name']:<30} - {bust['player_name']:<25} ({bust['pos']}, Rd {bust['round']}) Pick {bust['pick_no']:3}        {bust['total_points']:>6.2f} pts")




💔 WORST DRAFT PICKS (Biggest Busts)
────────────────────────────────────────────────────────────────────────────────────────────────────
   1. 🙌🏼 Praise Jah 🙌🏼               - Kaleb Johnson             (RB, Rd 8) Pick  77          0.10 pts
   2. Kenergy                        - Calvin Ridley             (WR, Rd 6) Pick  60         21.40 pts
   3. 🙌🏼 Praise Jah 🙌🏼               - T.J. Hockenson            (TE, Rd 9) Pick  84         24.00 pts
   4. 🙌🏼 Praise Jah 🙌🏼               - Mike Evans                (WR, Rd 4) Pick  37         27.00 pts
   5. Spicy Bijan Mustard            - James Conner              (RB, Rd 4) Pick  39         29.30 pts
   6. Spicy Bijan Mustard            - Matthew Golden            (WR, Rd 8) Pick  79         40.30 pts
   7. El Travador                    - Tyreek Hill               (WR, Rd 3) Pick  27         43.00 pts
   8. Jefferson Starship             - Cooper Kupp               (WR, Rd 10) Pick  96         45.30 pts
   9. Super Weenie Hut Jr           

In [77]:
# LATE-ROUND STEALS (Picks 101+ excluding QB, DEF, K)
print("\n\n🌟 LATE-ROUND STEALS (Picks 101+) - Skill Positions Only")
print("─" * 100)

late_round_steals_skill = []
for pick in draft_picks:
    if pick['pick_no'] and pick['pick_no'] > 100:
        player_id = pick['player_id']
        if player_id in players:
            player = players[player_id]
            pos = player.get('position', 'UNK')
            
            # Skip QB, DEF, and K
            if pos in ['QB', 'DEF', 'K']:
                continue
            
            player_name = player.get('full_name', f'Player {player_id}')
            total_points = player_total_points.get(player_id, 0)
            round_num = (pick['pick_no'] - 1) // 10 + 1
            
            # Get the team that drafted this player
            roster_id = pick.get('roster_id')
            team_name = "Unknown"
            if roster_id and roster_id in rosters_dict:
                roster = rosters_dict[roster_id]
                owner_id = roster.get('owner_id')
                if owner_id in users_dict:
                    team_name = users_dict[owner_id].get('metadata', {}).get('team_name', f'Team_{roster_id}')
            
            if total_points > 0:
                late_round_steals_skill.append({
                    'pick_no': pick['pick_no'],
                    'round': round_num,
                    'player_name': player_name,
                    'pos': pos,
                    'total_points': total_points,
                    'team_name': team_name
                })

late_round_steals_skill = sorted(late_round_steals_skill, key=lambda x: x['total_points'], reverse=True)
for i, steal in enumerate(late_round_steals_skill[:10], 1):
    print(f"  {i:2}. Pick {steal['pick_no']:3} (Rd {steal['round']:2}) | {steal['team_name']:<30} | {steal['player_name']:<25} ({steal['pos']}) | {steal['total_points']:>6.2f} pts")



🌟 LATE-ROUND STEALS (Picks 101+) - Skill Positions Only
────────────────────────────────────────────────────────────────────────────────────────────────────
   1. Pick 128 (Rd 13) | Egbuka Matata                  | Keenan Allen              (WR) | 119.90 pts
   2. Pick 101 (Rd 11) | Kenergy                        | Cam Skattebo              (RB) | 115.70 pts
   3. Pick 107 (Rd 11) | El Travador                    | Rashid Shaheed            (WR) | 106.40 pts
   4. Pick 123 (Rd 13) | Luke at my Big Dicker          | Kyle Pitts                (TE) | 100.10 pts
   5. Pick 106 (Rd 11) | Kittle Me This                 | Jauan Jennings            (WR) |  96.30 pts
   6. Pick 102 (Rd 11) | Spicy Bijan Mustard            | Jordan Addison            (WR) |  89.10 pts
   7. Pick 145 (Rd 15) | Jefferson Starship             | Woody Marks               (RB) |  85.00 pts
   8. Pick 112 (Rd 12) | Super Weenie Hut Jr            | Jacory Croskey-Merritt    (RB) |  83.10 pts
   9. Pick 126 (Rd 13) | 